In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [2]:
## Load the dataset
data=pd.read_csv("diabetes.csv")
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
import streamlit as st
threshold = data['DiabetesPedigreeFunction'].median()
data['FamilyHistory'] = data['DiabetesPedigreeFunction'].apply(lambda x: 1 if x >= 0.5 else 0)


In [4]:
data.drop(columns=['DiabetesPedigreeFunction'], inplace=True,axis=1)
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,Age,Outcome,FamilyHistory
0,6,148,72,35,0,33.6,50,1,1
1,1,85,66,29,0,26.6,31,0,0
2,8,183,64,0,0,23.3,32,1,1
3,1,89,66,23,94,28.1,21,0,0
4,0,137,40,35,168,43.1,33,1,1


In [5]:
print(pd.crosstab(data['FamilyHistory'], data['Outcome']))

Outcome          0    1
FamilyHistory          
0              349  142
1              151  126


In [6]:
## DiVide the dataset into indepent and dependent features
X=data.drop('Outcome',axis=1)
y=data['Outcome']

## Split the data in training and tetsing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [7]:
X_train

array([[-0.52639686, -1.15139792, -3.75268255, ..., -4.13525578,
        -1.03594038, -0.73406207],
       [ 1.58804586, -0.27664283,  0.68034485, ..., -0.48916881,
         1.48710085,  1.36228263],
       [-0.82846011,  0.56687102, -1.2658623 , ..., -0.42452187,
        -0.94893896,  1.36228263],
       ...,
       [ 1.8901091 , -0.62029661,  0.89659009, ...,  1.76054443,
         0.44308379,  1.36228263],
       [-1.13052335,  0.62935353, -3.75268255, ...,  1.34680407,
        -0.33992901, -0.73406207],
       [-1.13052335,  0.12949347,  1.43720319, ..., -1.22614383,
        -1.03594038, -0.73406207]])

In [8]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [9]:
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,Age,Outcome,FamilyHistory
0,6,148,72,35,0,33.6,50,1,1
1,1,85,66,29,0,26.6,31,0,0
2,8,183,64,0,0,23.3,32,1,1
3,1,89,66,23,94,28.1,21,0,0
4,0,137,40,35,168,43.1,33,1,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,63,0,0
764,2,122,70,27,0,36.8,27,0,0
765,5,121,72,23,112,26.2,30,0,0
766,1,126,60,0,0,30.1,47,1,0


# ANN IMPLEMENTATION

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [11]:
(X_train.shape[1],)

(8,)

In [12]:
## Build Our ANN Model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ## HL1 Connected wwith input layer
    Dense(32,activation='relu'), ## HL2
    Dense(1,activation='sigmoid')  ## output layer
]

)

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                576       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2689 (10.50 KB)
Trainable params: 2689 (10.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss

In [15]:
## compile the model
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [16]:
## Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [17]:
## Set up Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [18]:
### Train the model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


20/20 [==============================] - 3s 57ms/step - loss: 0.5338 - accuracy: 0.7329 - val_loss: 0.5519 - val_accuracy: 0.6948
Epoch 2/100
20/20 [==============================] - 0s 24ms/step - loss: 0.4542 - accuracy: 0.7769 - val_loss: 0.5350 - val_accuracy: 0.7532
Epoch 3/100
20/20 [==============================] - 0s 20ms/step - loss: 0.4396 - accuracy: 0.7850 - val_loss: 0.5210 - val_accuracy: 0.7727
Epoch 4/100
20/20 [==============================] - 0s 18ms/step - loss: 0.4168 - accuracy: 0.8062 - val_loss: 0.5527 - val_accuracy: 0.7597
Epoch 5/100
20/20 [==============================] - 0s 18ms/step - loss: 0.4160 - accuracy: 0.8013 - val_loss: 0.5344 - val_accuracy: 0.7208
Epoch 6/100
20/20 [==============================] - 0s 18ms/step - loss: 0.4135 - accuracy: 0.8094 - val_loss: 0.5532 - val_accuracy: 0.7403
Epoch 7/100
20/20 [==============================] - 0s 19ms/step - loss: 0.3967 - accuracy: 0.8225 - val_loss: 0.6071 - val_accuracy: 0.7208
Epoc

In [19]:
model.save('model.h5')

d:\Annie\DIABETES_PREDICTOR\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
## Load Tensorboard Extension
%reload_ext tensorboard

In [21]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6007 (pid 23100), started 2 days, 3:07:53 ago. (Use '!kill 23100' to kill it.)